In [1]:
! pip install --user --quiet psycopg2-binary
! pip install --user --quiet records
! pip install --user --quiet cockroachdb
! pip install --user --quiet toolz

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
import os

os.environ['CHATBASE_DATABASE'] =  "chatroach"
os.environ['CHATBASE_USER'] =  "chatroach"
os.environ['CHATBASE_PASSWORD'] =  ""
os.environ['CHATBASE_HOST'] =  "localhost"
os.environ['CHATBASE_PORT'] =  "5432"

In [4]:
BAILER_HOURS = 12
BAILER_DAYS = 5
FORMS = { 'eng': 'bailout_eng', 'hindi': 'bailout_hindi' }
BOTSERVER_URL = 'http://localhost:5000'
FB_PAGE_ID = '935593143497601'

In [7]:
import json
import time
import datetime
import os
import records
import pandas as pd
import numpy as np
import requests
from toolz import dissoc, get_in


def current_timestamp():
    return int(time.mktime(datetime.datetime.now().timetuple())) * 1000

# def get_times(df):
#     df = df.sort_values('timestamp')
#     args = np.argwhere((df.ref == 'QQQ').values)[:, 0]
#     timestamp = df.iloc[args[0]].timestamp
#     last_seen = df.iloc[args[-1]].timestamp
#     try:
#         next_seen = df.iloc[args[-1] + 1].timestamp
#     except IndexError:
#         next_seen = current_timestamp()

#     userid = df.userid.iloc[0]
#     lang = df.lang.iloc[0]

#     continuation = df.iloc[args[-1]:].shape[0]

#     return pd.DataFrame([{ 'userid': userid, 'lang': lang, 'time': timestamp, 'pause': next_seen - last_seen, 'continuation': continuation }])


# def get_blocked(df, hours):
#     df = df.copy()

#     previously_bailed = df[df['event_type'] == 'bailout'].userid.unique()
#     df = df[~df.userid.isin(previously_bailed)]
#     df = df[(df.source == 'messenger') & df.text.notna()].reset_index(drop=True)

#     engs = df.text.str.contains('Hey! Have a look at these videos') == True
#     hindis = df.text.str.contains('Kripaya is sandesh ko Messenger par apne mitron ko bhejen') == True

#     df.loc[engs, 'ref'] = 'QQQ'
#     df.loc[hindis, 'ref'] = 'QQQ'

#     eng_users = df[engs].userid.unique()
#     hindi_users = df[hindis].userid.unique()

#     eng_affected = df[df.userid.isin(eng_users)].reset_index(drop=True)
#     eng_affected['lang'] = 'eng'

#     hindi_affected = df[df.userid.isin(hindi_users)].reset_index(drop=True)
#     hindi_affected['lang'] = 'hindi'

#     affected = pd.concat([eng_affected, hindi_affected]).reset_index(drop=True).sort_values(['userid', 'timestamp'])
#     pauses = affected.groupby('userid').apply(get_times).reset_index(drop=True)
#     blocked = pauses[(pauses.pause > 1000*60*60*hours) & (pauses.continuation < 6)].reset_index(drop=True)

#     return blocked

# def get_bailouts(blocked, days):
#     blocked['bailout_time'] = blocked.time.map(lambda i: datetime.datetime.fromtimestamp(i/1000)) + datetime.timedelta(days=days)
#     return blocked[blocked['bailout_time'] < datetime.datetime.now()]

def conn_string():
    user = os.getenv('CHATBASE_USER')
    password = os.getenv('CHATBASE_PASSWORD')
    db = os.getenv('CHATBASE_DATABASE')
    host = os.getenv('CHATBASE_HOST')
    port = os.getenv('CHATBASE_PORT')
    return f'cockroachdb://{user}:{password}@{host}:{port}/{db}'


# def _bail(page, user, form):
#     return { 'event': {'type': 'bailout',
#                        'value': {'form': form}},
#              'user': user,
#              'page': page }

# def bailout(user, lang):
#     form = FORMS[lang]
#     page = FB_PAGE_ID
#     dat = _bail(page, user, form)
#     res = requests.post(f'{BOTSERVER_URL}/synthetic', json=dat)
#     return res

In [10]:
def get_ref(d):
    md = get_in(['message', 'metadata'], d)
    if md:
        return json.loads(md).get('ref')

def safe_loads(s):
    return json.loads(s) if s else None

def get_df(conn_string):
    db = records.Database(conn_string)
    dat = db.query('select * from messages order by timestamp desc')
    dat = (r.as_dict() for r in dat)
    dat = ({**json.loads(r['content']), 'userid': r['userid']} for r in dat)
    dat = (dissoc(d, 'recipient', 'sender') for d in dat)
    dat = ({**d, 'text': get_in(['message', 'text'], d), 'ref': get_ref(d)} for d in dat)
    dat = ({**d, 'metadata': get_in(['message', 'metadata'], d)} for d in dat)
    dat = ({**d, 'is_echo': get_in(['message', 'is_echo'], d)} for d in dat)
    dat = ({**d, 'metadata': safe_loads(d['metadata'])} for d in dat)
    dat = ({**d, 'wait': get_in(['metadata', 'wait', 'value'], d)} for d in dat)
    dat = (dissoc(d, 'message', 'referral', 'user', 'page', 'postback', 'event', 'data', 'metadata') for d in dat)
    return pd.DataFrame(list(dat))


In [11]:
df = get_df(conn_string())

In [53]:
def is_waiting(df):
    df = df.sort_values('timestamp')
    args = np.argwhere(((df.wait == '168 hours') | (df.wait == '192 hours')).values)[:, 0]

    if not len(args):
        return None

    timestamp = df.iloc[args[0]].timestamp
    last_seen = df.iloc[args[-1]].timestamp
    # lang = df.lang.iloc[0]
    userid = df.userid.iloc[0]
    continuation = df.iloc[args[-1]:].shape[0]

    return df.assign(continuation = continuation)
    # final = df.sort_values('timestamp').iloc[-1]
    # if final.wait == '168 hours' or final.wait == '192 hours':
        # return True
    # return False

In [54]:
dd = df.groupby('userid').apply(is_waiting)

In [56]:
dd[dd['continuation'] < 3]

is_echo     source  \
userid                                        
1503630593093973 1635385    None  messenger   
                 1635353    True  messenger   
                 1635298    None  messenger   
                 1635285    True  messenger   
                 1635264    True  messenger   
...                          ...        ...   
4226115810747630 60833      None  synthetic   
                 60831      True  messenger   
                 60832      None  synthetic   
                 60829      True  messenger   
                 60830      None  synthetic   

                                                                       text  \
userid                                                                        
1503630593093973 1635385                                               None   
                 1635353  Hi Saba Khan, in which language would you like...   
                 1635298                                              Hindi   
                 1635285                                        Dhanyavaad!   
                 1635264  Aapaka svaagat hai Saba Khan! 😀 \nHum bharat m...   
...                                                                     ...   
4226115810747630 60833                                                 None   
                 60831    Thank you for your time!  \nIn the coming days...   
                 60832                                                 None   
                 60829                                           Thank you!   
                 60830                                                 None   

                              timestamp       wait  \
userid                                               
1503630593093973 1635385  1582021787252       None   
                 1635353  1582021795188       None   
                 1635298  1582021804270       None   
                 1635285  1582021807109       None   
                 1635264  1582021810670       None   
...                                 ...        ...   
4226115810747630 60833    1583039963877       None   
                 60831    1583039964400       None   
                 60832    1583039964799       None   
                 60829    1583039965346  192 hours   
                 60830    1583039965991       None   

                                                           ref  \
userid                                                           
1503630593093973 1635385                                  None   
                 1635353  09a14047-26f1-46b6-9c5f-cb7b1330cbad   
                 1635298                                  None   
                 1635285  f761c804-0a8e-4328-a29d-b9dec849f284   
                 1635264  000dd1a3-ea0e-43eb-97b3-4d10c66f1274   
...                                                        ...   
4226115810747630 60833                                    None   
                 60831    e3115214-f795-4561-8205-f9d708fecb70   
                 60832                                    None   
                 60829    895b5352-0c9b-4dc0-a38b-644f9bfc79a4   
                 60830                                    None   

                                    userid optin message_request  continuation  
userid                                                                          
1503630593093973 1635385  1503630593093973   NaN             NaN             2  
                 1635353  1503630593093973   NaN             NaN             2  
                 1635298  1503630593093973   NaN             NaN             2  
                 1635285  1503630593093973   NaN             NaN             2  
                 1635264  1503630593093973   NaN             NaN             2  
...                                    ...   ...             ...           ...  
4226115810747630 60833    4226115810747630   NaN             NaN             2  
                 60831    4226115810747630   NaN             NaN             2  
             

In [48]:
dd.userid.unique()

array(['1501574496633221', '1503630593093973', '1510287932428690', ...,
       '4441890412488489', '5134128719949112', '5334908873293683'],
      dtype=object)

In [116]:
df = get_df(conn_string())
blocked = get_blocked(df, BAILER_HOURS)
bails = get_bailouts(blocked, BAILER_DAYS)